## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City_ID.2,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,0,0,Rikitea,PF,-23.1203,-134.9692,73.20,69,62,20.51,broken clouds
1,1,1,1,Ahipara,NZ,-35.1667,173.1667,55.51,58,13,12.68,few clouds
2,2,2,2,Cancun,MX,21.1743,-86.8466,82.33,83,20,4.61,few clouds
3,3,3,3,Egvekinot,RU,66.3167,-179.1667,51.01,80,100,4.00,overcast clouds
4,4,4,4,Nizhniy Tsasuchey,RU,50.5167,115.1333,71.67,43,27,13.98,scattered clouds


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City_ID.2,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,0,0,Rikitea,PF,-23.1203,-134.9692,73.20,69,62,20.51,broken clouds
2,2,2,2,Cancun,MX,21.1743,-86.8466,82.33,83,20,4.61,few clouds
4,4,4,4,Nizhniy Tsasuchey,RU,50.5167,115.1333,71.67,43,27,13.98,scattered clouds
5,5,5,5,East London,ZA,-33.0153,27.9116,63.88,42,51,4.74,broken clouds
6,6,6,6,Owando,CG,-0.4819,15.8999,73.62,62,71,3.62,broken clouds
8,8,8,8,Dwarka,IN,22.2394,68.9678,82.13,87,100,9.08,overcast clouds
9,9,9,9,Bethel,US,41.3712,-73.4140,77.36,79,88,4.16,overcast clouds
10,10,10,10,Tari,NG,12.4086,4.4914,72.21,88,90,4.36,overcast clouds
15,15,15,15,Komsomolskiy,UZ,40.4272,71.7189,74.12,40,0,6.91,clear sky
19,19,19,19,Vyshkov,RU,52.4764,31.6873,62.73,85,5,8.25,clear sky


In [29]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                482
City_ID.2              482
City_ID.1              482
City                   482
Country                482
Lat                    482
Lng                    482
Max Temp               482
Humidity               482
Cloudiness             482
Wind Speed             482
Current Description    482
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows found

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,
2,Cancun,MX,82.33,few clouds,21.1743,-86.8466,
4,Nizhniy Tsasuchey,RU,71.67,scattered clouds,50.5167,115.1333,
5,East London,ZA,63.88,broken clouds,-33.0153,27.9116,
6,Owando,CG,73.62,broken clouds,-0.4819,15.8999,
8,Dwarka,IN,82.13,overcast clouds,22.2394,68.9678,
9,Bethel,US,77.36,overcast clouds,41.3712,-73.4140,
10,Tari,NG,72.21,overcast clouds,12.4086,4.4914,
15,Komsomolskiy,UZ,74.12,clear sky,40.4272,71.7189,
19,Vyshkov,RU,62.73,clear sky,52.4764,31.6873,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping to next city.")   
        

Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to next city.
Hotel not found... skipping to nex

In [50]:
hotel_df = hotel_df.replace(r'^s*$', float('NaN'), regex = True)
clean_hotel_df = hotel_df.dropna(subset = ["Hotel Name"])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,People ThankYou
2,Cancun,MX,82.33,few clouds,21.1743,-86.8466,Hotel Tankah Cancun
5,East London,ZA,63.88,broken clouds,-33.0153,27.9116,Tu Casa
6,Owando,CG,73.62,broken clouds,-0.4819,15.8999,Exaunel Hotel Owando
8,Dwarka,IN,82.13,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
...,...,...,...,...,...,...,...
679,Sigiriya,LK,82.24,scattered clouds,7.9518,80.7426,Hotel Sigiriya
681,Noshiro,JP,77.83,overcast clouds,40.2039,140.0242,Suigetsu
683,Partenit,UA,73.58,clear sky,44.5780,34.3446,Приватотель
685,Tamandare,BR,73.35,clear sky,-8.7597,-35.1047,Pousada 100 Passos


In [51]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Database.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))